In [1]:
from pprint import pprint
from elasticsearch import Elasticsearch

es = Elasticsearch('http://localhost:9200')
client_info = es.info()
print('Connected to Elasticsearch!')
pprint(client_info.body)

Connected to Elasticsearch!
{'cluster_name': 'docker-cluster',
 'cluster_uuid': '_jy6LMJiQFudVV9QHpDdWg',
 'name': 'ef46826769b8',
 'tagline': 'You Know, for Search',
 'version': {'build_date': '2025-05-28T10:06:37.834829258Z',
             'build_flavor': 'default',
             'build_hash': '0a58bc1dc7a4ae5412db66624aab968370bd44ce',
             'build_snapshot': False,
             'build_type': 'docker',
             'lucene_version': '10.1.0',
             'minimum_index_compatibility_version': '8.0.0',
             'minimum_wire_compatibility_version': '8.18.0',
             'number': '9.0.2'}}


In [15]:
es.indices.delete(index='test')

ObjectApiResponse({'acknowledged': True})

In [16]:
#list files
try:
    indices = list(es.indices.get_alias(index="*").keys())
    
    indices_reais = [idx for idx in indices if not idx.startswith('.')]
    print("Índices encontrados:", indices_reais)
except Exception as e:
    print(f"Erro ao tentar obter os índices: {e}")

Índices encontrados: []


In [14]:
#list documents
index_name = 'test'

try:
    response = es.count(index=index_name)
    total = response['count']
    print(f"Total documents in '{index_name}': {total}")
except Exception as e:
    print(f"Error counting documents: {e}")

Total documents in 'test': 1
